# 과제 1. 학습 루프 이해 (개념)<br>
**Q. 딥러닝 학습 루프(forward → loss → zero_grad → backward → step)의 각 단계가 어떤 일을 수행하는지 한 줄씩 설명하시오.**<br><br>
A.
1. forward: 예측값(pred) 계산
2. loss: 오차 측정
3. zero_grad: 이전 기울기 초기화
4. backward: 역전파(기울기 계산)
5. step: 파라미터 업데이트

# 과제 2. 선형회귀 학습 구현<br>
**Q. 아래와 같은 데이터셋을 생성하고, PyTorch의 Autograd를 활용하여 선형회귀를 학습시키시오.**<br>
조건:  
* 반복 횟수는 1000회
* learning rate는 0.1
* loss는 MSE, optimizer는 BGD를 사용할 것
* **단, torch.nn, torch.optim는 사용금지(MSE, BGD는 직접 구현)**
* 선형회귀 식이 어떻게 될지 잘 생각해보고 결과가 어떤 형태여야할지 잘 생각해볼 것



In [112]:
import torch

print(torch.__version__)

2.5.1


In [113]:
torch.manual_seed(0) # 랜덤 시드 고정
x = torch.rand(100, 2)
y = 3 * x + 2 + 0.1 * torch.randn(100, 2)
print(x.shape, y.shape)

torch.Size([100, 2]) torch.Size([100, 2])


In [114]:
epochs = 1000
lr = 0.1
w = torch.randn(2, 2, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w.size(), b.size())

torch.Size([2, 2]) torch.Size([2])


In [115]:
for epoch in range(epochs):
    
    pred = x.matmul(w) + b

    loss = 0.0
    for i in range(len(pred)):
        for j in range(pred.size(1)):
            loss += (pred[i][j] - y[i][j])**2
    
    loss /= len(pred)*2

    loss.backward()

    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad

        w.grad.zero_()
        b.grad.zero_()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 100/1000, Loss: 0.0538
Epoch 200/1000, Loss: 0.0217
Epoch 300/1000, Loss: 0.0136
Epoch 400/1000, Loss: 0.0115
Epoch 500/1000, Loss: 0.0110
Epoch 600/1000, Loss: 0.0109
Epoch 700/1000, Loss: 0.0108
Epoch 800/1000, Loss: 0.0108
Epoch 900/1000, Loss: 0.0108
Epoch 1000/1000, Loss: 0.0108


In [116]:
print(f'Learned parameters: \nw = {w.detach().numpy()}, \n\nb = {b.detach().numpy()}')

Learned parameters: 
w = [[ 3.0510375  -0.03443779]
 [ 0.0065079   2.9894495 ]], 

b = [1.9775236 1.999864 ]


# 과제 3. Pytorch 모델 구조로 변환<br>
**Q. 위 과제 2를 동일하게 구현하되, PyTorch의 nn.Linear, nn.MSELoss, torch.optim.SGD를 이용해 학습시키시오.**  
* 위에서는 BGD를 사용했으나, 여기선 SGD를 사용함을 유의할 것


In [117]:
import torch.nn as nn
import torch.optim as optim

In [118]:
torch.manual_seed(0) # 랜덤 시드 고정
x = torch.rand(100, 2)
y = 3 * x + 2 + 0.1 * torch.randn(100, 2)
print(x.shape, y.shape)

torch.Size([100, 2]) torch.Size([100, 2])


In [119]:
epochs = 1000
lr = 0.1
w = torch.randn(2, 2, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w.size(), b.size())

torch.Size([2, 2]) torch.Size([2])


In [120]:
model = nn.Linear(2, 2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 100/1000, Loss: 0.1029
Epoch 200/1000, Loss: 0.0339
Epoch 300/1000, Loss: 0.0166
Epoch 400/1000, Loss: 0.0123
Epoch 500/1000, Loss: 0.0112
Epoch 600/1000, Loss: 0.0109
Epoch 700/1000, Loss: 0.0109
Epoch 800/1000, Loss: 0.0108
Epoch 900/1000, Loss: 0.0108
Epoch 1000/1000, Loss: 0.0108


In [121]:
print(f'Learned parameters: \nw = {model.weight.detach().numpy()}, \n\nb = {model.bias.detach().numpy()}')

Learned parameters: 
w = [[ 3.0503006   0.00688288]
 [-0.03443708  2.988854  ]], 

b = [1.9776987 2.0001705]


# 4. 과제 4. DataSet, DataLoader 실습
**Q. 아래와 같은 데이터셋을 생성하고, TensorDataset과 DataLoader를 활용하여 배치로 나눈 뒤 선형회귀를 학습시키시오.**<br>
조건:  
* 반복 횟수는 100회
* learning rate는 0.1
* loss는 MSE, optimizer는 SGD를 사용할 것
* batch_size는 16, shufle은 True

In [122]:
from torch.utils.data import TensorDataset, DataLoader

In [123]:
torch.manual_seed(0) # 랜덤 시드 고정
x = torch.rand(100, 2)
y = 3 * x + 2 + 0.1 * torch.randn(100, 2)
print(x.shape, y.shape)

torch.Size([100, 2]) torch.Size([100, 2])


In [124]:
dataset = TensorDataset(x, y)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [125]:
epochs = 100
lr = 0.1
w = torch.randn(2, 2, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w.size(), b.size())

torch.Size([2, 2]) torch.Size([2])


In [126]:
model = nn.Linear(2, 2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    for x_batch, y_batch in loader:
        optimizer.zero_grad()
        pred = model(x_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 10/100, Loss: 0.1450
Epoch 20/100, Loss: 0.0647
Epoch 30/100, Loss: 0.0677
Epoch 40/100, Loss: 0.0396
Epoch 50/100, Loss: 0.0153
Epoch 60/100, Loss: 0.0039
Epoch 70/100, Loss: 0.0091
Epoch 80/100, Loss: 0.0234
Epoch 90/100, Loss: 0.0127
Epoch 100/100, Loss: 0.0181


In [127]:
print(f'Learned parameters: \nw = {model.weight.detach().numpy()}, \n\nb = {model.bias.detach().numpy()}')

Learned parameters: 
w = [[ 3.0395734   0.02055518]
 [-0.02583804  2.9748273 ]], 

b = [1.9906256 2.0020266]
